In [0]:
dbutils.widgets.text("env", "aws")
dbutils.widgets.text("storage_root", "s3://pakeyj-data-sales")
dbutils.widgets.text("process_date", "2026-01-17")
print("env:", dbutils.widgets.get("env"))
print("storage_root:", dbutils.widgets.get("storage_root"))
print("process_date:", dbutils.widgets.get("process_date"))

In [0]:
from src.common.config import load_config

cfg = load_config()
storage_root = cfg.storage_root
process_date = cfg.process_date
orders_out = cfg.paths["bronze_orders_daily"]
items_out  = cfg.paths["bronze_order_items_daily"]
meta_out   = f"{storage_root}/bronze/_generator_runs"

In [0]:


def peek(df, name, n=10):
    print(f"\n=== {name} ===")
    print(f"rows: {df.count()}")
    df.limit(n).show(truncate=False)

df = spark.read.json(orders_out) 
peek(df, "orders_out")
print("test")